In [ ]:
"""
   Copyright 2021-2022 Boris Shminke

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
"""

In [1]:
from glob import glob
import os
import shutil

def remove_dirs() -> None:
    for path in glob(os.path.join(".", "hyp*")):
        shutil.rmtree(path)
    for path in glob(os.path.join(".", "task*")):
        shutil.rmtree(path)

In [2]:
# check that any lattice axiom is independent from all the rest
from residuated_binars.constants import LATTICE
from residuated_binars.use_nitpick import use_nitpick

remove_dirs()
server_info = (
    'server "test" = 127.0.0.1:36245 (password "2dfdb3fb-4de2-4c20-9270-95fadd652880")'
)
max_dim = 10
use_nitpick(
    max_cardinality=max_dim,
    independent_assumptions=[
        # this is needed to skip the lattice axioms check
        # what we are trying to find here are not real lattices
        axiom.replace("join", "joan")
        for axiom in LATTICE
    ],
    additional_assumptions=[],
    check_subset_independence=False,
    server_info=server_info
)

In [3]:
# in this package we have also a parser for Isabelle server responses
import os
from residuated_binars.parser import isabelle_response_to_algebra

structures = []
for i in range(2, max_dim):
    try:
        structures += isabelle_response_to_algebra(
            os.path.join(".", f"task{i}", "isabelle.out")
        )
    except:
        pass

In [4]:
from residuated_binars.axiom_checkers import commutative, associative, absorbs

for some in structures:
    score = ""
    score += "1" if commutative(some.operations["joan"]) else "0"
    score += "1" if commutative(some.operations["meet"]) else "0"
    score += "1" if associative(some.operations["joan"]) else "0"
    score += "1" if associative(some.operations["meet"]) else "0"
    score += "1" if absorbs(some.operations["meet"], some.operations["joan"]) else "0"
    score += "1" if absorbs(some.operations["joan"], some.operations["meet"]) else "0"
    print(score)

    

111110
111101
011111
101111
110111
111011
